In [19]:
import json
import geojson
import random

After `infrastructure-issue-identifier.ipynb` identifies issues from the raw 311 Service Requests, it must be preprocessed to be displayed on an interactive map.

This will consist of repackaging the raw, augmented 311 Service Request JSON generated by `infrastructure-issue-identifier.ipynb` into a GeoJSON FeatureCollection that can be displayed.

In [20]:
# Define the names of the required files here:
RAW_JSON_FILE_NAME = 'data/infra_issues.json'
GEOJSON_FILE_NAME = 'data/infra_issues.geojson'

In [21]:
# Function to process single 311 Service Request JSON to a GeoJson Feature
def process_service_request(service_request:dict, layer:int=0) -> geojson.Feature:
    # Define the properties of the feature (description, address, request datetime, assigned class, keyword_matches, media URL)
    properties_dict = {}
    properties_dict['Address'] = service_request.get('address', 'No address provided.')
    properties_dict['Date/Time'] = service_request.get('requested_datetime', 'Unknown')
    properties_dict['Description'] = service_request.get('description', 'No description provided.')
    properties_dict['Type of Issue'] = service_request.get('class', 'No class assigned.')
    properties_dict['Keyword Matches'] = service_request.get('keyword_matches', 'No keyword matches found.')
    properties_dict['Attached Media'] = service_request.get('media_url', 'No media attached.')

    # Define the geometry for the feature
    # Add some randomness into the lattitude and longitude values to prevent exact overlap of service requests
    layer_offset = layer * 5e-5
    lattitude = service_request['lat'] + layer_offset
    longitude = service_request['long'] + layer_offset
    geometry = geojson.Point((longitude, lattitude))

    # Construct the Feature
    return geojson.Feature(geometry=geometry, properties=properties_dict, id=service_request['service_request_id'])

In [24]:
# Import the augmented 311 service request JSON
raw_issues_json = json.loads(open(RAW_JSON_FILE_NAME, 'r').read())

# Structure of GeoJSON file will be a single feature collection
# Keep track of seen addresses to have multiple reports at same address stack nicely
times_seen_address = {}
feature_list = []
for request in raw_issues_json:
    address = request.get('address', None)
    layer = times_seen_address.get(address, -1)
    layer+=1
    times_seen_address[address] = layer
    processed_request = process_service_request(request, layer)
    feature_list.append(processed_request)
# feature_list = [process_service_request(req) for req in raw_issues_json]


# Create the FeatureCollection
feature_collection = geojson.FeatureCollection(feature_list)

# Write the FeatureCollection to the given file
with open(GEOJSON_FILE_NAME, 'w') as file:
    json.dump(feature_collection, file)